In [21]:
import torchvision
from torch.utils.tensorboard import SummaryWriter
import torch
torch.cuda.is_available()

True

In [22]:
import torch

# 1. 检查CUDA是否可用
print("CUDA可用:", torch.cuda.is_available())  # 应返回True

# 2. 查看显卡数量和名称
print("显卡数量:", torch.cuda.device_count())  # 至少为1
print("当前显卡:", torch.cuda.current_device())  # 通常为0
print("显卡名称:", torch.cuda.get_device_name(0))  # 显示具体型号

# 3. 验证CUDA计算（创建张量并在GPU上运算）
if torch.cuda.is_available():
    # 在GPU上创建一个随机张量
    x = torch.rand(3, 3).cuda()
    # 执行简单计算（矩阵乘法）
    y = x @ x.T  # 等价于x.matmul(x.T)
    print("GPU计算结果:\n", y)

CUDA可用: True
显卡数量: 1
当前显卡: 0
显卡名称: NVIDIA GeForce RTX 3060 Laptop GPU
GPU计算结果:
 tensor([[1.6030, 0.9837, 1.6295],
        [0.9837, 0.9157, 0.9851],
        [1.6295, 0.9851, 1.7168]], device='cuda:0')


In [23]:
#准备数据集
from torch import nn
from torch.utils.data import DataLoader

train_data = torchvision.datasets.MNIST(root="../data",train = True, transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.MNIST(root="../data",train = False, transform=torchvision.transforms.ToTensor(),download=True)

#数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集长度：{}".format(train_data_size))
print("测试数据集长度：{}".format(test_data_size))



训练数据集长度：60000
测试数据集长度：10000


In [24]:

#加载数据集
train_dataloader = DataLoader(train_data, batch_size=8192)
test_dataloader = DataLoader(test_data, batch_size=8192)

In [25]:
#创建网络
class LeNet(nn.Module):
    def __init__(self) :
        super(LeNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,6,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5,120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84,10)

    #前向传播函数
    def forward(self,x):
            x = self.conv1(x)
            x = self.conv2(x)
            x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, 16 * 5 * 5)
            x = self.fc1(x)
            x = self.fc2(x)
            output = self.fc3(x)

            return output

In [26]:
myLeNet = LeNet()
myLeNet = myLeNet.cuda()

if torch.cuda.is_available():	# cuda可用时可置于GPU中训练网络，以加快计算速度
    myLeNet.cuda()
    print("cuda is available, and the calculation will be moved to GPU\n")
else:
    print("cuda is unavailable!")


cuda is available, and the calculation will be moved to GPU



In [27]:
#训练和测试
optimizer = torch.optim.RMSprop(myLeNet.parameters(),lr = 1e-3)
loss_func = nn.CrossEntropyLoss()
loss_func = loss_func.cuda()

In [28]:
#设置训练的一些参数
total_train_step = 0
total_test_step = 0
opoch = 50

In [29]:
#添加Tensorboard
writer = SummaryWriter("../logs_train")

In [30]:
for i in range(opoch):
    print("------第{}轮训练开始------".format(i+1))    
#训练开始
    myLeNet.train()
    for data in train_dataloader:
        imgs, target = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            target = target.cuda()
        outputs = myLeNet(imgs)
        loss = loss_func(outputs, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数:{}, Loss:{}".format(total_train_step,loss.item()))
            writer.add_scalar("train_loss",loss.item(),total_train_step)
    #测试步骤
    myLeNet.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,target = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                target = target.cuda()
            outputs =  myLeNet(imgs)
            loss = loss_func(outputs, target)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == target).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试Loss:{}".format(total_test_loss))
    print("整体测试准确率:{}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,total_test_step)
    total_test_step = total_test_step + 1
    torch.save(myLeNet,"myLeNet.pth".format(i))
    print("模型已保存")

------第1轮训练开始------
整体测试Loss:3.6201884746551514
整体测试准确率:0.29589998722076416
模型已保存
------第2轮训练开始------
整体测试Loss:1.388041913509369
整体测试准确率:0.7910999655723572
模型已保存
------第3轮训练开始------
整体测试Loss:0.9339116811752319
整体测试准确率:0.8531999588012695
模型已保存
------第4轮训练开始------
整体测试Loss:0.7786813676357269
整体测试准确率:0.8633999824523926
模型已保存
------第5轮训练开始------
整体测试Loss:0.5938539952039719
整体测试准确率:0.9021999835968018
模型已保存
------第6轮训练开始------
整体测试Loss:0.5828786790370941
整体测试准确率:0.890999972820282
模型已保存
------第7轮训练开始------
整体测试Loss:0.45774662494659424
整体测试准确率:0.9286999702453613
模型已保存
------第8轮训练开始------
整体测试Loss:0.4013039767742157
整体测试准确率:0.9325000047683716
模型已保存
------第9轮训练开始------
整体测试Loss:0.43490926921367645
整体测试准确率:0.9297999739646912
模型已保存
------第10轮训练开始------
整体测试Loss:0.3350471407175064
整体测试准确率:0.9418999552726746
模型已保存
------第11轮训练开始------
整体测试Loss:0.32094739377498627
整体测试准确率:0.9431999921798706
模型已保存
------第12轮训练开始------
整体测试Loss:0.3023948594927788
整体测试准确率:0.9467999935150146
模型已保存
------第13轮训练开始------
训练

In [31]:

writer.close()